In [0]:
# Defyning variables to connect with db
driver = "org.postgresql.Driver"
database_host = "psql-mock-database-cloud.postgres.database.azure.com"
database_port = "5432" # update if you use a non-default port
database_name = "ecom1689953593876hxincshnhoiqouxc"
user = "leimsgqkzowufdayhgewwwrf@psql-mock-database-cloud"
password = "dujntukagjbgkgkuejjjjcik"

In [0]:
# uri connection with db
url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"

In [0]:
import pyspark
from pyspark.sql.functions import col, asc,desc, sum, regexp_replace

In [0]:
# Questão 1

# bringing table orders
table = "orders"
df_orders = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

# bringing table orderdetails
table = "orderdetails"
df_orderdetails = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

# bringing table customers
table = "customers"
df_customers = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)


In [0]:
df_question_1 = df_orders.join(df_orderdetails, on='order_number',how='inner')
df_question_1 = df_question_1.join(df_customers, how='inner',on='customer_number')
df_question_1 = df_question_1.filter(df_question_1["status"]=='Cancelled')
df_question_1 = df_question_1.select('country','quantity_ordered').groupBy('country').sum('quantity_ordered')
df_question_1.orderBy(col('sum(quantity_ordered)').desc()).show()
# RESPOSTA: Espanha foi o país com maior quantidade de items cancelados

+-----------+---------------------+
|    country|sum(quantity_ordered)|
+-----------+---------------------+
|      Spain|                  605|
|New Zealand|                  596|
|     Sweden|                  550|
|        USA|                  454|
|         UK|                  429|
+-----------+---------------------+



In [0]:
# Questão 2

# bringing table orders
table = "orders"
df_orders = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

# bringing table orderdetails
table = "orderdetails"
df_orderdetails = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

# bringing table products
table = "products"
df_products = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

In [0]:
df_question_2 = df_orders.join(df_orderdetails, on='order_number',how='inner')
df_question_2 = df_question_2.join(df_products, how='inner',on='product_code')
df_question_2 = df_question_2.filter((col('order_date').startswith('2005')) & (col('status')=='Shipped'))
grouped = df_question_2.select('product_code','product_name','price_each','quantity_ordered').groupBy('product_code').sum('quantity_ordered').orderBy(col('sum(quantity_ordered)').desc())
grouped.select('product_code')
#S18_3232 is the best seller
df_question_2 = df_question_2.filter(col('product_code')=='S18_3232').select('product_code','product_name','price_each','quantity_ordered')
df_question_2 = df_question_2.withColumn('total', df_question_2.price_each * df_question_2.quantity_ordered)
df_question_2 = df_question_2.select(sum(df_question_2.total))
# Resposta: O produto mais vendido foi a 1992 Ferrari Red com faturamento de $52978.28 

In [0]:
df_question_2.show()

+----------+
|sum(total)|
+----------+
|  52978.28|
+----------+



In [0]:
# Questão 3

# bringing table offices
table = "offices"
df_offices = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

# bringing table employees
table = "employees"
df_employees = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

In [0]:
df_question_3 = df_offices.join(df_employees, on='office_code',how='inner')
df_question_3 = df_question_3.filter(df_question_3["country"]=='Japan').select('first_name','last_name','email')
df_question_3 = df_question_3.select('first_name','last_name',regexp_replace('email','(^[^@]*)','*'))
df_question_3.show()

+----------+---------+-------------------------------------+
|first_name|last_name|regexp_replace(email, (^[^@]*), *, 1)|
+----------+---------+-------------------------------------+
|      Mami|    Nishi|                 *@classicmodelcar...|
|   Yoshimi|     Kato|                 *@classicmodelcar...|
+----------+---------+-------------------------------------+



In [0]:
df_question_1 = df_question_1.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in df_question_1.columns]).orderBy(col('sum(quantity_ordered)').desc())

df_question_2 = df_question_2.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in df_question_2.columns])

df_question_3 = df_question_3.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in df_question_3.columns])

In [0]:
df_question_1.write.mode("overwrite").format("delta").saveAsTable('question1_pyspark')
df_question_2.write.mode("overwrite").format("delta").saveAsTable('question2_pyspark')
df_question_3.write.mode("overwrite").format("delta").saveAsTable('question3_pyspark')